In [0]:
rest_api_key = "pjonp7jywyzz2hxqiex7lrls1vqkmjfn/"
interactive_map_api_key = "e3b3l6eu4dt57q7o5hu2ks7knw5mfvl6/"

In [2]:
## DEMO URL CREATION 

# rest_api_key = "pjonp7jywyzz2hxqiex7lrls1vqkmjfn/"
# interactive_map_api_key = "e3b3l6eu4dt57q7o5hu2ks7knw5mfvl6/"

# request_dist_matrix_URL = "https://apis.mapmyindia.com/advancedmaps/v1/"+ rest_api_key + "distance_matrix/driving/77.5998448,12.5090914;77.5800417,12.5092973?dep_time=1531543500"
# request_dist_URL = "https://apis.mapmyindia.com/advancedmaps/v1/"+ rest_api_key + "distance?center=28.54589623,77.285698%7C&pts=29,78%7C30,78%7C28,79"

baseURL = "https://apis.mapmyindia.com/advancedmaps/v1/"
startCoord = "77.5998448,12.5090914;"
Coord1 = "77.5800417,12.5092973"
Coord2 = "75.9800417,15.7092973"

url = baseURL+rest_api_key+"distance_matrix/"+"driving/"+startCoord+Coord1#+Coord2
print(url)


https://apis.mapmyindia.com/advancedmaps/v1/jry2mpzyiykhd78e23mmjxr4nc59kwhf/distance_matrix/driving/77.5998448,12.5090914;77.5800417,12.5092973


In [3]:
## DEMO DATA RETREIVAL (API CALL)

# !pip install requests
import requests
import json

# resp = requests.get('https://apis.mapmyindia.com/advancedmaps/v1/pjonp7jywyzz2hxqiex7lrls1vqkmjfn/snapToRoad?pts=78.40573,17.37317;78.40958,17.37314;78.4 1845,17.37449;78.409992,17.37328;78.420460,17.377443;78.421350,17.380200×tamps=1527056019;1527056020;1 527056021;1527056022;1527056023;1527056024')
resp = requests.get(url)

if resp.status_code != 200:
  print("[INFO] Error: "+str(resp.status_code))
else :
    y=resp.json()
    print(y["results"])

{'code': 'Ok', 'distances': [[0, 3038.2]], 'durations': [[0, 674.2]]}


In [0]:
## DRIVE ACCESS
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir("drive/My Drive/Colab Notebooks/KTJ2020")
!ls

MapMyIndia_RestAPI.ipynb  modify_csv.ipynb  Petrol-Pumps.csv


In [0]:
# ## READ CSV (lat,long,profit)
# import pandas as pd
# col_name = ["Index","Latitute","Longitute"]
# coords = pd.read_csv("Petrol-Pumps.csv", index_col=0, header=None,delimiter=";", names=col_name)
# coords.head()
# # coords.to_csv("Petrol-Pumps_new.csv")
# # coords = pd.read_csv("Petrol-Pumps_new.csv", index_col=0, header=0)
# # coords.head()

# ## ADD PROFIT TO CSV
# import random
# profits = [random.randrange(1, 100, 1) for i in range(coords.shape[0])]
# coords["Profit"]=profits
# # coords.drop(labels=None,columns=Index)
# coords.to_csv("Petrol-Pumps_new.csv",index=False)
# coords = pd.read_csv("Petrol-Pumps_new.csv")
# coords.head()

In [0]:
## READING LAT-LONG_PROFIT
import pandas as pd
coords= pd.read_csv("Petrol-Pumps.csv")
coords.head()

,Latitute,Longitute,Profit
0,28.746731,77.151751,48
1,28.544869,77.128134,69
2,28.698757,77.292577,17
3,28.675332,77.225378,40
4,28.597172,77.195823,92


In [0]:
def make_request(url):
  resp = requests.get(url)
  if resp.status_code != 200:
    print("[INFO] Error: "+str(resp.status_code))
  else :
    y=resp.json()
    return (y["results"]["distances"][0][1])


In [0]:
## GET DISTANCE ROW FOR EACH STARTING POINT
def get_distance(coords,start_point=0,point_count=20):
  distanceList=[]
  url = baseURL+rest_api_key+"distance_matrix/"+"driving/"
  startLat=coords.at[start_point,'Latitute']
  startLong=coords.at[start_point,'Longitute']
  url+=str(startLong)+","+str(startLat)
  for index,row in coords.iterrows():
      lati = row["Latitute"]
      longi = row["Longitute"]
      tempUrl=url+";"+str(longi)+","+str(lati)
      distanceList.append(make_request(tempUrl))
      
      if index>point_count:
        break


  # print(count)
  return distanceList

In [0]:
## GET DISTANCE OF ALL POINTS FROM STARTING POINT
def get_distance_from_start(coords,start_point=0,point_count=20):
  distanceList=[]
  url = baseURL+rest_api_key+"distance_matrix/"+"driving/"
  startLat=28.80 #coords.at[start_point,'Latitute']
  startLong=77.35 #coords.at[start_point,'Longitute']
  url+=str(startLong)+","+str(startLat)
  for index,row in coords.iterrows():
      lati = row["Latitute"]
      longi = row["Longitute"]
      tempUrl=url+";"+str(longi)+","+str(lati)
      distanceList.append(make_request(tempUrl))
      
      if index>point_count:
        break


  # print(count)
  import csv
  with open('distance_from_start.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(distanceList)

  return distanceList

In [0]:
# get_distance(coords,10)

In [0]:
## CREATE A DISTANCE MATRIX BETWEEN ALL POINTS
def distance_matrix(coords, point_count=20):
  allDistances = []
  for index,row in coords.iterrows():
    allDistances.append(get_distance(coords,index,point_count))

    if index>point_count:
      break

  dist_matrix = pd.DataFrame(allDistances)
  return dist_matrix

In [0]:
## CALL THE FUNCTIONS
dist_matrix = distance_matrix(coords, point_count = 50)
dist_matrix.to_csv("distance_matrix.csv",index=False)
startDistance = get_distance_from_start(coords,point_count=50)

In [0]:
#### RUN ROUTE OPTIMIZATION CODE BY ATUL ######